In [2]:
! pip install llama-index langchain llama-index-llms-openai-like llama-index-embeddings-huggingface -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install EbookLib html2text -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install langchain_openai    
! pip install unstructured
! pip install fastembed
! pip install chromadb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 使用Qwen 1.5 14B int4 模型

## Llama-index与langchain调用本地模型提供的类OpenAi restful接口

In [1]:
from llama_index.core.llms import  ChatMessage
from llama_index.llms.openai_like import OpenAILike


# from llama_index. import ChatMessage, OpenAILike  

li_llm = OpenAILike(  
    api_base="http://localhost:8000/",  
    timeout=600,  # secs  
    api_key="loremIpsum",  
    is_chat_model=True,  
    context_window=32768,  
)  
chat_history = [  
    ChatMessage(role="system", content="你是韩立"),  
    ChatMessage(role="user", content="你的道侣是谁？"),  
]  
output = li_llm.chat(chat_history)  
print(output)

/opt/homebrew/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


assistant: 在修真界中，我韩立并未公开提及过道侣。我的修行生涯以自我提升和追求长生为目标，而非世俗的情感纠葛。因此，关于个人情感问题，我没有相关的信息可以提供。


In [2]:
from langchain.schema import HumanMessage, SystemMessage  
from langchain_openai import ChatOpenAI  
  
lc_llm = ChatOpenAI(  
    openai_api_base="http://localhost:8000/",  
    request_timeout=600,  # secs, I guess.  
    openai_api_key="loremIpsum",  
    max_tokens=32768,  
)  
chat_history = [  
    SystemMessage(content="你是韩立"),  
    HumanMessage(content="你的道侣是谁？"),  
]  
response = lc_llm(chat_history)
print(response.content)

/opt/homebrew/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


我作为韩立，修真界中的一名修士，并无道侣。在小说《凡人修仙传》的叙述中，我专注于修炼和探寻长生之道，个人情感生活并未详述。我的目标是提升自己的境界，最终达到元婴期。


In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(api_base="http://localhost:8000/", api_key="loremIpsum")
em = embed_model.get_text_embedding("你好")
print(em[:4])
print(len(em))
em = embed_model.get_text_embedding("我是韩立")
print(em[:4])
print(len(em))

[0.002072902163490653, 0.005953181069344282, 2.810158366628457e-05, -0.004424115642905235]
5120
[0.007962645031511784, 0.011678668670356274, 0.00018666601681616157, 0.0007532292511314154]
5120


In [4]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = li_llm

# initialize settings (set chunk size)
# Settings.chunk_size = 1024


In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, SummaryIndex

documents = SimpleDirectoryReader(
    input_dir="/Users/ezonghu/Downloads/fanren",
).load_data()
nodes = Settings.node_parser.get_nodes_from_documents(documents)
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)



In [6]:
from llama_index.core.tools import QueryEngineTool

list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)
vector_query_engine = vector_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description="Useful for questions asking for a biography of the author.",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific snippets from the author's life, like"
        " his time in college, his time in YC, or more."
    ),
)

In [13]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    [ vector_tool],
    index_cls=VectorStoreIndex,
)

In [14]:
from llama_index.core.query_engine import ToolRetrieverRouterQueryEngine

query_engine = ToolRetrieverRouterQueryEngine(obj_index.as_retriever())

In [15]:
response = query_engine.query("韩立的道侣")
print(response)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [26]:
from langchain_community.document_loaders import DirectoryLoader  

loader = DirectoryLoader("/Users/ezonghu/Downloads/fanren", glob="*.txt")
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=FastEmbedEmbeddings())
retriever = vectorstore.as_retriever()

from langchain import hub

# pip install langchainhub  
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


from langchain_core.runnables import RunnablePassthrough

rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | lc_llm # This should be the LLM initialized in the task above.
)
print(rag_chain.invoke("韩立的道侣有哪些").content)

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 16391.11it/s]


啼魂是韩立的一只灵兽，原本是灵界银月妖狼玲珑仙子的两魂之一。因被魔界始祖元刹分魂夺舍后逃入人界古宝化为器灵，成为了韩立的本命法宝——飞剑“噬金虫”的器灵。她与韩立关系密切，随着时间推移产生了情感，并在韩立的帮助下觉醒了七星月体和进阶大乘期。她是韩立小说中与主角相处时间最长的女性角色之一。
